In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("clean_total.csv")

In [3]:
df.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,booking_date,release_date,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age
0,679537,301831.0,1974-04-02,MALE,WHITE,M,SHAWNEE,KS,66226.0,15CR00773,...,2015-04-01,2015-04-01,CA,10000.0,1,"MAJOR DEPRESSIVE DISORDER, SINGLE EPISODE, MOD...",OTHERS,1,0,41.0
1,431835,296410.0,1971-01-01,MALE,WHITE,S,OLATHE,KS,66062.0,13CR00129,...,2013-01-19,2013-01-22,SUR,10000.0,1,SOCIAL PHOBIA,OTHERS,1,1,42.0
2,431835,296410.0,1971-01-01,MALE,WHITE,S,OLATHE,KS,66062.0,14CR00932,...,2014-05-02,2014-05-02,SUR,5000.0,1,SOCIAL PHOBIA,OTHERS,1,1,43.0
3,658145,350849.0,1997-04-12,FEMALE,WHITE,S,EDGERTON,KS,66021.0,15CR02335,...,2015-10-07,2015-10-07,SUR,10000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,18.0
4,655865,370683.0,1984-08-03,MALE,WHITE,S,OLATHE,KS,66062.0,13CR00468,...,2013-03-03,2013-03-03,SUR,1000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,29.0


In [4]:
test = df.copy()

In [5]:
test = test[test['re_entry']==1]

In [6]:
test['booking_date'] = pd.to_datetime(test['booking_date'])

In [7]:
test['release_date'] = pd.to_datetime(test['release_date'])

In [8]:
def within_frame(df, id_num='dedupe_id', timestamp='booking_date', col_name='re-entry', duration=365):
	df = df.sort_values(by=[id_num, timestamp])
	df['{}'.format(col_name)] = df.groupby(id_num)[timestamp].diff()
	df['{}'.format(col_name)] = df['{}'.format(col_name)].apply(lambda x: x.days)
	df['within_{}'.format(duration)] = np.where(df['{}'.format(col_name)]>duration, 1, 0)    
	
	return df

In [9]:
test = test.sort_values(by=['dedupe_id', 'booking_date'])

In [10]:
test.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,booking_date,release_date,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age
20701,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR01499,...,2011-07-05,2011-07-05,PR,2500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,21.0
20702,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR02750,...,2011-11-20,2012-10-22,SUR,2500.0,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,22.0
20703,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR01019,...,2014-05-02,2014-05-19,SUR,1500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0
20704,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR02622,...,2014-12-03,2014-12-04,SUR,5000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0
10703,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,14DV00270,...,2014-03-04,2014-03-05,PR,1500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,36.0


In [11]:
test['1yr'] = test.groupby('dedupe_id')['booking_date'].diff()

In [12]:
test['stayed'] = test['release_date'] - test['booking_date']

In [13]:
test['stayed'].head()

20701     0 days
20702   337 days
20703    17 days
20704     1 days
10703     1 days
Name: stayed, dtype: timedelta64[ns]

In [14]:
test['1yr'].head()

20701        NaT
20702   138 days
20703   894 days
20704   215 days
10703        NaT
Name: 1yr, dtype: timedelta64[ns]

In [15]:
test[['mni_no', '1yr', 'stayed']].head()

,mni_no,1yr,stayed
20701,494625,NaT,0 days
20702,494625,138 days,337 days
20703,494625,894 days,17 days
20704,494625,215 days,1 days
10703,673675,NaT,1 days


In [16]:
test.stayed = test.stayed.shift(1)

In [17]:
test[['mni_no', '1yr', 'stayed']].head()

,mni_no,1yr,stayed
20701,494625,NaT,NaT
20702,494625,138 days,0 days
20703,494625,894 days,337 days
20704,494625,215 days,17 days
10703,673675,NaT,1 days


In [18]:
test = test[test['1yr'].notnull()]

In [19]:
test[['mni_no', '1yr', 'stayed']].head()

,mni_no,1yr,stayed
20702,494625,138 days,0 days
20703,494625,894 days,337 days
20704,494625,215 days,17 days
10704,673675,387 days,1 days
10705,673675,335 days,1 days


In [20]:
test['after_released'] = test['1yr']-test['stayed']

In [21]:
test.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age,1yr,stayed,after_released
20702,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR02750,...,2500.0,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,22.0,138 days,0 days,138 days
20703,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR01019,...,1500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,894 days,337 days,557 days
20704,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR02622,...,5000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,215 days,17 days,198 days
10704,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,15DV00382,...,1000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,37.0,387 days,1 days,386 days
10705,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,16DV00234,...,2500.0,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,38.0,335 days,1 days,334 days


In [22]:
#Convert the days type to integer 
test['after_released'] = test['after_released'].astype('timedelta64[D]')

In [23]:
test.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age,1yr,stayed,after_released
20702,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR02750,...,2500.0,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,22.0,138 days,0 days,138.0
20703,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR01019,...,1500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,894 days,337 days,557.0
20704,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR02622,...,5000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,215 days,17 days,198.0
10704,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,15DV00382,...,1000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,37.0,387 days,1 days,386.0
10705,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,16DV00234,...,2500.0,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,38.0,335 days,1 days,334.0


In [24]:
test["within_one"] = np.where(test["after_released"] <= 365, 1, 0)

In [25]:
test.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age,1yr,stayed,after_released,within_one
20702,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR02750,...,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,22.0,138 days,0 days,138.0,1
20703,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR01019,...,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,894 days,337 days,557.0,0
20704,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR02622,...,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,215 days,17 days,198.0,1
10704,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,15DV00382,...,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,37.0,387 days,1 days,386.0,0
10705,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,16DV00234,...,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,38.0,335 days,1 days,334.0,1


In [26]:
test.to_csv('final_data.csv', index=False)

In [27]:
test.columns

Index(['mni_no', 'dedupe_id', 'dob', 'sex', 'race', 'mar_stat', 'city',
       'state', 'zip', 'case_no', 'case_type', 'booking_no',
       'arresting_agency', 'arresting_agency_type', 'booking_date',
       'release_date', 'bail_type', 'bail_amt', 'bailed_out', 'pri_dx_value',
       'refferal_source', 'mh_treatment', 're_entry', 'age', '1yr', 'stayed',
       'after_released', 'within_one'],
      dtype='object')